In [12]:
import pandas as pd

# Đọc dữ liệu từ tệp CSV
data = pd.read_csv('../../../data/gdp.csv')
# data = pd.read_csv('../../../data/BCP.csv')
# Xem trước dữ liệu
print(data.head())
print(data.info())


   Year         GDP
0  1985  235.652540
1  1986  430.188501
2  1987  585.304937
3  1988  397.014360
4  1989   96.130362
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    38 non-null     int64  
 1   GDP     38 non-null     float64
dtypes: float64(1), int64(1)
memory usage: 740.0 bytes
None


# Cài đặt thuật toán Logistics

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score

# Đọc dữ liệu từ tệp CSV mới được tải lên
try:
    data = pd.read_csv('../../../data/gdp.csv')

    # Xem trước dữ liệu
    data_head = data.head()
    data_columns = data.columns

    # Giả sử cột 'GDP' là mục tiêu và các cột còn lại là đặc trưng
    target_column = 'GDP'
    features = [col for col in data.columns if col != target_column]

    # Chuyển đổi GDP thành nhị phân (0 hoặc 1) dựa trên giá trị trung bình
    data['GDP_binary'] = (data[target_column] > data[target_column].mean()).astype(int)

    X = data[features]
    y = data['GDP_binary']

    # Chia dữ liệu thành tập huấn luyện và tập kiểm tra
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Chuyển đổi X_test thành DataFrame để sử dụng thuộc tính index
    X_test_df = pd.DataFrame(X_test, columns=features)

    # Chuẩn hóa dữ liệu
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Huấn luyện mô hình hồi quy Logistic
    logistic_model = LogisticRegression()
    logistic_model.fit(X_train, y_train)

    # Dự đoán với mô hình hồi quy Logistic
    y_pred_logistic = logistic_model.predict(X_test)

    # Huấn luyện mô hình MLP
    mlp_model = MLPClassifier(hidden_layer_sizes=(10,), max_iter=1000, random_state=42)
    mlp_model.fit(X_train, y_train)

    # Dự đoán với mô hình MLP
    y_pred_mlp = mlp_model.predict(X_test)

    # Kết hợp kết quả dự đoán từ cả hai mô hình
    y_pred_combined = (y_pred_logistic + y_pred_mlp) / 2
    y_pred_combined = np.round(y_pred_combined).astype(int)

    # Đánh giá độ chính xác và F1 score
    accuracy = accuracy_score(y_test, y_pred_combined)
    f1 = f1_score(y_test, y_pred_combined)

    # Thêm cột dự đoán vào dữ liệu ban đầu
    data.loc[X_test_df.index, 'target_prediction'] = y_pred_combined

    # Hiển thị 10 dòng đầu tiên của dữ liệu kết quả
    data_result = data.head(10)

    print("Data Head:")
    print(data_head)
    print("\nData Columns:")
    print(data_columns)
    print("\nAccuracy:", accuracy)
    print("F1 Score:", f1)
    print("\nData Result:")
    print(data_result)

except Exception as e:
    print(f"Error processing the data: {e}")


Data Head:
   Year         GDP
0  1985  235.652540
1  1986  430.188501
2  1987  585.304937
3  1988  397.014360
4  1989   96.130362

Data Columns:
Index(['Year', 'GDP'], dtype='object')

Accuracy: 0.875
F1 Score: 0.8888888888888888

Data Result:
   Year         GDP  GDP_binary  target_prediction
0  1985  235.652540           0                NaN
1  1986  430.188501           0                NaN
2  1987  585.304937           0                NaN
3  1988  397.014360           0                NaN
4  1989   96.130362           0                0.0
5  1990   96.719296           0                NaN
6  1991  140.631004           0                0.0
7  1992  141.383685           0                NaN
8  1993  185.187122           0                NaN
9  1994  224.637094           0                NaN


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
